# Factorizations and other fun
Based on work by Andreas Noack

## Outline
 - Factorizations
 - Special matrix structures
 - Generic linear algebra

Before we get started, let's set up a linear system and use `LinearAlgebra` to bring in the factorizations and special matrix structures.

In [51]:
using LinearAlgebra
A = rand(3, 3)
x = fill(1, (3,))
b = A * x

3-element Vector{Float64}:
 1.5092028074042236
 1.6033387820086238
 1.7118709139315202

## Factorizations

#### LU factorizations
In Julia we can perform an LU factorization
```julia
PA = LU
``` 
where `P` is a permutation matrix, `L` is lower triangular unit diagonal and `U` is upper triangular, using `lufact`.

Julia allows computing the LU factorization and defines a composite factorization type for storing it.

In [52]:
Alu = lu(A)

LU{Float64, Matrix{Float64}, Vector{Int64}}
L factor:
3×3 Matrix{Float64}:
 1.0       0.0       0.0
 0.777928  1.0       0.0
 0.919413  0.364101  1.0
U factor:
3×3 Matrix{Float64}:
 0.574916  0.202584   0.825839
 0.0       0.616375  -0.151787
 0.0       0.0       -0.134084

In [53]:
typeof(Alu)

LU{Float64, Matrix{Float64}, Vector{Int64}}

The different parts of the factorization can be extracted by accessing their special properties

In [54]:
Alu.P

3×3 Matrix{Float64}:
 0.0  1.0  0.0
 0.0  0.0  1.0
 1.0  0.0  0.0

In [55]:
Alu.L

3×3 Matrix{Float64}:
 1.0       0.0       0.0
 0.777928  1.0       0.0
 0.919413  0.364101  1.0

In [56]:
Alu.U

3×3 Matrix{Float64}:
 0.574916  0.202584   0.825839
 0.0       0.616375  -0.151787
 0.0       0.0       -0.134084

Julia can dispatch methods on factorization objects.

For example, we can solve the linear system using either the original matrix or the factorization object.

In [57]:
A\b

3-element Vector{Float64}:
 1.0
 0.9999999999999998
 1.0

In [58]:
Alu\b

3-element Vector{Float64}:
 1.0
 0.9999999999999998
 1.0

Similarly, we can calculate the determinant of `A` using either `A` or the factorization object

In [59]:
det(A) ≈ det(Alu)

true

#### QR factorizations

In Julia we can perform a QR factorization
```
A=QR
``` 

where `Q` is unitary/orthogonal and `R` is upper triangular, using `qrfact`. 

In [60]:
Aqr = qr(A)

LinearAlgebra.QRCompactWY{Float64, Matrix{Float64}, Matrix{Float64}}
Q factor:
3×3 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}, Matrix{Float64}}:
 -0.587332  -0.0673872  -0.806536
 -0.638812  -0.573284    0.513092
 -0.49695    0.816581    0.293661
R factor:
3×3 Matrix{Float64}:
 -0.899977  -0.755244  -1.10613
  0.0        0.488197  -0.111186
  0.0        0.0        0.108144

Similarly to the LU factorization, the matrices `Q` and `R` can be extracted from the QR factorization object via

In [61]:
Aqr.Q

3×3 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}, Matrix{Float64}}:
 -0.587332  -0.0673872  -0.806536
 -0.638812  -0.573284    0.513092
 -0.49695    0.816581    0.293661

In [62]:
Aqr.R

3×3 Matrix{Float64}:
 -0.899977  -0.755244  -1.10613
  0.0        0.488197  -0.111186
  0.0        0.0        0.108144

#### Eigendecompositions

The results from eigendecompositions, singular value decompositions, Hessenberg factorizations, and Schur decompositions are all stored in `Factorization` types.

The eigendecomposition can be computed

In [63]:
Asym = A + A'
AsymEig = eigen(Asym)

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
3-element Vector{Float64}:
 -0.9400690436689478
  0.14798028222992077
  3.23573995974025
vectors:
3×3 Matrix{Float64}:
  0.0746564   0.835471  -0.544439
 -0.78542    -0.287139  -0.548331
  0.614445   -0.46855   -0.634759

The values and the vectors can be extracted from the Eigen type by special indexing

In [64]:
AsymEig.values

3-element Vector{Float64}:
 -0.9400690436689478
  0.14798028222992077
  3.23573995974025

In [65]:
AsymEig.vectors

3×3 Matrix{Float64}:
  0.0746564   0.835471  -0.544439
 -0.78542    -0.287139  -0.548331
  0.614445   -0.46855   -0.634759

Once again, when the factorization is stored in a type, we can dispatch on it and write specialized methods that exploit the properties of the factorization, e.g. that $A^{-1}=(V\Lambda V^{-1})^{-1}=V\Lambda^{-1}V^{-1}$.

In [66]:
inv(AsymEig)*Asym

3×3 Matrix{Float64}:
 1.0          -4.44089e-15   1.59872e-14
 3.33067e-15   1.0          -1.02141e-14
 5.10703e-15  -8.88178e-16   1.0

## Special matrix structures
Matrix structure is very important in linear algebra. To see *how* important it is, let's work with a larger linear system

In [67]:
n = 1000
A = randn(n,n);

Julia can often infer special matrix structure

In [68]:
Asym = A + A'
issymmetric(Asym)

true

but sometimes floating point error might get in the way.

In [69]:
Asym_noisy = copy(Asym)
Asym_noisy[1,2] += 5eps()

-0.9576907072457186

In [70]:
issymmetric(Asym_noisy)

false

Luckily we can declare structure explicitly with, for example, `Diagonal`, `Triangular`, `Symmetric`, `Hermitian`, `Tridiagonal` and `SymTridiagonal`.

In [71]:
Asym_explicit = Symmetric(Asym_noisy);

Let's compare how long it takes Julia to compute the eigenvalues of `Asym`, `Asym_noisy`, and `Asym_explicit`

In [72]:
@time eigvals(Asym);

  0.201886 seconds (1.65 k allocations: 8.095 MiB, 1.85% compilation time)


In [73]:
@time eigvals(Asym_noisy);

  0.575040 seconds (14 allocations: 7.928 MiB)


In [74]:
@time eigvals(Asym_explicit);

  0.252358 seconds (69.33 k allocations: 12.700 MiB, 3.12% gc time, 20.33% compilation time)


In this example, using `Symmetric()` on `Asym_noisy` made our calculations about `5x` more efficient :)

#### A big problem
Using the `Tridiagonal` and `SymTridiagonal` types to store tridiagonal matrices makes it possible to work with potentially very large tridiagonal problems. The following problem would not be possible to solve on a laptop if the matrix had to be stored as a (dense) `Matrix` type.

In [75]:
n = 1_000_000;
A = SymTridiagonal(randn(n), randn(n-1));
@time eigmax(A)

  0.488004 seconds (920.23 k allocations: 238.831 MiB, 3.26% gc time, 38.61% compilation time)


6.547504493867262

## Generic linear algebra
The usual way of adding support for numerical linear algebra is by wrapping BLAS and LAPACK subroutines. For matrices with elements of `Float32`, `Float64`, `Complex{Float32}` or `Complex{Float64}` this is also what Julia does.

However, Julia also supports generic linear algebra, allowing you to, for example, work with matrices and vectors of rational numbers.

#### Rational numbers
Julia has rational numbers built in. To construct a rational number, use double forward slashes:

In [76]:
1//2

1//2

#### Example: Rational linear system of equations
The following example shows how linear system of equations with rational elements can be solved without promoting to floating point element types. Overflow can easily become a problem when working with rational numbers so we use `BigInt`s.

In [77]:
Arational = Matrix{Rational{BigInt}}(rand(1:10, 3, 3))/10

3×3 Matrix{Rational{BigInt}}:
 1//5  3//10  1//5
 3//5  3//10  4//5
 1//2  1//10  3//5

In [78]:
x = fill(1, 3)
b = Arational*x

3-element Vector{Rational{BigInt}}:
  7//10
 17//10
  6//5

In [79]:
Arational\b

3-element Vector{Rational{BigInt}}:
 1//1
 1//1
 1//1

In [80]:
lu(Arational)

LU{Rational{BigInt}, Matrix{Rational{BigInt}}, Vector{Int64}}
L factor:
3×3 Matrix{Rational{BigInt}}:
 1//1   0//1  0//1
 1//3   1//1  0//1
 5//6  -3//4  1//1
U factor:
3×3 Matrix{Rational{BigInt}}:
 3//5  3//10   4//5
 0//1  1//5   -1//15
 0//1  0//1   -7//60

### Exercises

#### 11.1
What are the eigenvalues of matrix A?

```
A =
[
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36
]
```
and assign it a variable `A_eigv`

In [81]:
using LinearAlgebra

In [82]:
A =
[
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36
]

A_eigv = eigvals(A)

5-element Vector{Float64}:
 -128.49322764802145
  -55.887784553056875
   42.7521672793189
   87.16111477514521
  542.4677301466143

In [83]:
@assert A_eigv ==  [-128.49322764802145, -55.887784553056875, 42.7521672793189, 87.16111477514521, 542.4677301466143]

#### 11.2 
Create a `Diagonal` matrix from the eigenvalues of `A`.

In [84]:
A_diag = Diagonal(A_eigv)

5×5 Diagonal{Float64, Vector{Float64}}:
 -128.493     ⋅        ⋅        ⋅         ⋅ 
     ⋅     -55.8878    ⋅        ⋅         ⋅ 
     ⋅        ⋅      42.7522    ⋅         ⋅ 
     ⋅        ⋅        ⋅      87.1611     ⋅ 
     ⋅        ⋅        ⋅        ⋅      542.468

In [85]:
A_diag = zeros(length(A_eigv), length(A_eigv))

for i in 1:length(A_eigv)
    A_diag[i,i] = round(A_eigv[i], digits = 4)
end

A_diag

5×5 Matrix{Float64}:
 -128.493    0.0      0.0      0.0       0.0
    0.0    -55.8878   0.0      0.0       0.0
    0.0      0.0     42.7522   0.0       0.0
    0.0      0.0      0.0     87.1611    0.0
    0.0      0.0      0.0      0.0     542.468

The answer must be:

```
[-128.493    0.0      0.0      0.0       0.0;
    0.0    -55.8878   0.0      0.0       0.0;
    0.0      0.0     42.7522   0.0       0.0;
    0.0      0.0      0.0     87.1611    0.0;
    0.0      0.0      0.0      0.0     542.468]
```


#### 11.3 
Create a `LowerTriangular` matrix from `A` and store it in `A_lowertri`

In [86]:
A_lowertri = LowerTriangular(A)

5×5 LowerTriangular{Int64, Matrix{Int64}}:
 140    ⋅    ⋅    ⋅   ⋅
  97  106    ⋅    ⋅   ⋅
  74   89  152    ⋅   ⋅
 168  131  144   54   ⋅
 131   36   71  142  36

The points are zeros. Just to remember, we can access to columns and rows in this way:

In [87]:
A_lowertri[:,1] #column 1

5-element Vector{Int64}:
 140
  97
  74
 168
 131

In [88]:
A_lowertri[1,:] #row 1

5-element Vector{Int64}:
 140
   0
   0
   0
   0

In [89]:
@assert A_lowertri ==  [140    0    0    0   0;
  97  106    0    0   0;
  74   89  152    0   0;
 168  131  144   54   0;
 131   36   71  142  36]